In [1]:
import numpy as np

In [3]:
N = 100_000
sum = 0
tot = 0

for _ in range(N):
    r = min(np.random.uniform(0,1), np.random.uniform(0,1))
    tot += 1
    sum += r

print(sum/tot)

0.3322325131058587


In [33]:
np.median([0, 1.5,1.51])

1.5

In [34]:
0.25*0.25*0.75*3 + 0.25**3

0.15625

In [37]:
5/16

0.3125